This file briefly explains how the data was formatted

# 1) Setup

## 1a) Library functions

In [2]:
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from time import time
from datetime import datetime, timedelta

print("done")

done


## 1b) Helper functions

In [18]:
def ListUnion(l1, l2):
    union = list(set(l1) | set(l2))
    return union

# 2) SPX Index Members
Uses a web scraper from https://github.com/leosmigel/analyzingalpha/tree/master/2019-09-18-sp500-historical-components-and-changes
- Note: only data until Sep 2021 was collected
- Did not rerun scraper since we only consider prices up to Dec 2021

In [40]:
df_CurrentMembers = pd.read_csv("MarketData/SPX_CurrentMembers.txt", encoding = "utf-16")
df_CurrentMembers = df_CurrentMembers.iloc[:,1:]
df_HistoricalMembers = pd.read_csv("MarketData/SPX_HistoricalMembers.txt", encoding = "utf-16")
df_HistoricalMembers = df_HistoricalMembers.iloc[:,1:]

tickers_current = list(df_CurrentMembers["ticker"])
tickers_historical = list(df_HistoricalMembers["ticker"])
tickers_all = ListUnion(tickers_current,tickers_historical)
print(len(tickers_all))
with open("MarketData/SPX_AllMembers.txt", 'w') as f:
    for item in tickers_all:
        f.write("%s\n" % item)
print("done")

774
done


In [41]:
df_CurrentMembers

,ticker,name,date,cik
0,MMM,3M,1976-08-09,66740
1,ABT,Abbott Laboratories,1964-03-31,1800
2,ABBV,AbbVie,2012-12-31,1551152
3,ABMD,Abiomed,2018-05-31,815094
4,ACN,Accenture,2011-07-06,1467373
...,...,...,...,...
500,YUM,Yum! Brands,1997-10-06,1041061
501,ZBRA,Zebra Technologies,2019-12-23,877212
502,ZBH,Zimmer Biomet,2001-08-07,1136869
503,ZION,Zions Bancorp,2001-06-22,109380


# 3) Analyst Estimates
- Downloaded from WRDS: https://wrds-www.wharton.upenn.edu/pages/get-data/ibes-thomson-reuters/ibes-academic/unadjusted-summary/price-target/
- Note: sometimes few analyst estimates, so median may be more reliable than direction

In [75]:
df_AnalystEstimates = pd.read_csv("FundamentalData/AnalystEstimates.csv")
df_AnalystEstimates

,OFTIC,TICKER,CUSIP,CNAME,STATPERS,NUMEST,NUMUP4W,NUMDOWN4W,NUMUP1M,NUMDOWN1M,MEANPTG,MEDPTG,STDEV,PTGHIGH,PTGLOW,CURR
0,A,@F9I,SI496526,ALFESCA,2006/03/16,1,0,0,0,0,5.100,5.1,NaN,5.1,5.1,ISK
1,A,@F9I,SI496526,ALFESCA,2006/04/20,1,1,0,1,0,5.400,5.4,NaN,5.4,5.4,ISK
2,A,@F9I,SI496526,ALFESCA,2006/05/18,1,0,0,0,0,5.400,5.4,NaN,5.4,5.4,ISK
3,A,@F9I,SI496526,ALFESCA,2006/06/15,1,0,0,0,0,5.400,5.4,NaN,5.4,5.4,ISK
4,A,@F9I,SI496526,ALFESCA,2006/07/20,1,0,0,0,0,5.400,5.4,NaN,5.4,5.4,ISK
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
228342,ZTS,ZOTS,98978V10,ZOETIS INC,2021/08/19,13,9,0,9,0,225.000,227.0,14.259,240.0,184.0,USD
228343,ZTS,ZOTS,98978V10,ZOETIS INC,2021/09/16,13,0,0,0,0,225.000,227.0,14.259,240.0,184.0,USD
228344,ZTS,ZOTS,98978V10,ZOETIS INC,2021/10/14,12,0,0,0,0,224.583,226.0,14.811,240.0,184.0,USD
228345,ZTS,ZOTS,98978V10,ZOETIS INC,2021/11/18,11,7,0,7,0,236.545,245.0,21.398,258.0,184.0,USD


In [50]:
#median price targets
df_temp = df_AnalystEstimates[["OFTIC", "STATPERS","MEDPTG"]]
df_AnalystMedian = pd.pivot_table(df_temp, values='MEDPTG', index=["STATPERS"],
                    columns=['OFTIC'])
df_AnalystMedian

OFTIC,A,AA,AAL,AAP,AAPL,ABBV,ABC,ABK,ABMD,ABS,...,XRAY,XRX,XTO,XYL,YHOO,YUM,ZBH,ZBRA,ZION,ZTS
STATPERS,,,,,,,,,,,,,,,,,,,,,
2000/01/20,80.0,100.0,NaN,NaN,135.00,NaN,NaN,77.0,NaN,45.50,...,36.5,38.0,17.0,NaN,495.0,61.0,NaN,60.0,74.0,NaN
2000/02/17,85.0,100.0,NaN,NaN,137.00,NaN,NaN,71.0,NaN,45.00,...,36.5,35.0,17.0,NaN,247.5,61.0,NaN,75.0,72.0,NaN
2000/03/16,172.5,97.5,NaN,NaN,139.00,NaN,NaN,74.0,100.0,44.00,...,36.5,35.0,17.5,NaN,250.0,61.0,NaN,77.5,69.5,NaN
2000/04/20,160.0,95.2,NaN,NaN,140.00,NaN,NaN,75.0,100.0,41.00,...,36.0,35.0,17.0,NaN,250.0,60.0,NaN,80.0,58.0,NaN
2000/05/18,125.0,95.0,NaN,NaN,140.00,NaN,NaN,75.0,100.0,35.00,...,37.0,32.5,20.0,NaN,247.0,50.0,NaN,80.0,58.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021/08/19,170.0,50.0,35.8435,230.0,168.00,126.0,526.100000,NaN,415.0,23.65,...,74.5,20.0,NaN,130.0,NaN,138.0,193.5,595.0,57.0,227.0
2021/09/16,170.0,51.0,35.6565,242.5,169.64,127.5,526.500000,NaN,415.0,23.65,...,74.5,20.0,NaN,130.0,NaN,138.0,193.5,605.0,59.0,227.0
2021/10/14,176.0,56.0,33.9365,240.5,169.64,129.5,526.500000,NaN,415.0,23.65,...,72.0,18.0,NaN,125.0,NaN,138.0,189.0,617.5,64.0,226.0


In [54]:
# %up
df_temp = df_AnalystEstimates[["OFTIC", "STATPERS","NUMUP1M","NUMEST"]]
df_temp["NUMUP1M"] = df_temp["NUMUP1M"]/df_temp["NUMEST"]
df_AnalystUp = pd.pivot_table(df_temp, values='NUMUP1M', index=["STATPERS"],
                    columns=['OFTIC'])
df_AnalystUp

C:\Users\pyliu\AppData\Local\Temp/ipykernel_22392/3874097912.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_temp["NUMUP1M"] = df_temp["NUMUP1M"]/df_temp["NUMEST"]


OFTIC,A,AA,AAL,AAP,AAPL,ABBV,ABC,ABK,ABMD,ABS,...,XRAY,XRX,XTO,XYL,YHOO,YUM,ZBH,ZBRA,ZION,ZTS
STATPERS,,,,,,,,,,,,,,,,,,,,,
2000/01/20,0.428571,0.583333,NaN,NaN,0.384615,NaN,NaN,0.0,NaN,0.00,...,0.000000,0.000000,0.0,NaN,0.636364,0.000000,NaN,0.000000,0.142857,NaN
2000/02/17,0.285714,0.000000,NaN,NaN,0.000000,NaN,NaN,0.0,NaN,0.00,...,0.000000,0.142857,0.0,NaN,0.000000,0.000000,NaN,0.500000,0.000000,NaN
2000/03/16,0.750000,0.000000,NaN,NaN,0.000000,NaN,NaN,0.0,0.0,0.00,...,0.000000,0.000000,0.0,NaN,0.000000,0.000000,NaN,0.250000,0.000000,NaN
2000/04/20,0.000000,0.071429,NaN,NaN,0.214286,NaN,NaN,0.2,0.0,0.00,...,0.000000,0.000000,0.0,NaN,0.090909,0.000000,NaN,0.250000,0.285714,NaN
2000/05/18,0.000000,0.076923,NaN,NaN,0.000000,NaN,NaN,0.0,0.0,0.00,...,0.000000,0.000000,0.6,NaN,0.000000,0.000000,NaN,0.000000,0.000000,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021/08/19,0.812500,0.454545,0.353535,0.266667,0.600000,0.375000,0.351140,NaN,0.5,0.25,...,0.083333,0.600000,NaN,0.833333,NaN,0.826087,0.038462,0.700000,0.095238,0.692308
2021/09/16,0.117647,0.545455,0.052139,0.750000,0.142857,0.090909,0.215456,NaN,0.0,0.00,...,0.000000,0.000000,NaN,0.000000,NaN,0.045455,0.000000,0.200000,0.285714,0.000000
2021/10/14,0.125000,0.454545,0.000000,0.000000,0.000000,0.045455,0.051282,NaN,0.0,0.00,...,0.000000,0.000000,NaN,0.090909,NaN,0.000000,0.000000,0.200000,0.409091,0.000000


In [55]:
# %down
df_temp = df_AnalystEstimates[["OFTIC", "STATPERS","NUMDOWN1M","NUMEST"]]
df_temp["NUMDOWN1M"] = df_temp["NUMDOWN1M"]/df_temp["NUMEST"]
df_AnalystDown = pd.pivot_table(df_temp, values='NUMDOWN1M', index=["STATPERS"],
                    columns=['OFTIC'])
df_AnalystDown

C:\Users\pyliu\AppData\Local\Temp/ipykernel_22392/3173791829.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_temp["NUMDOWN1M"] = df_temp["NUMDOWN1M"]/df_temp["NUMEST"]


OFTIC,A,AA,AAL,AAP,AAPL,ABBV,ABC,ABK,ABMD,ABS,...,XRAY,XRX,XTO,XYL,YHOO,YUM,ZBH,ZBRA,ZION,ZTS
STATPERS,,,,,,,,,,,,,,,,,,,,,
2000/01/20,0.000,0.000000,NaN,NaN,0.000,NaN,NaN,0.000000,NaN,0.000000,...,0.000000,0.0,0.000000,NaN,0.000000,0.000000,NaN,0.0,0.000000,NaN
2000/02/17,0.000,0.083333,NaN,NaN,0.000,NaN,NaN,0.200000,NaN,0.000000,...,0.000000,0.0,0.222222,NaN,0.000000,0.166667,NaN,0.0,0.000000,NaN
2000/03/16,0.000,0.142857,NaN,NaN,0.000,NaN,NaN,0.000000,0.0,0.222222,...,0.000000,0.0,0.000000,NaN,0.090909,0.000000,NaN,0.0,0.285714,NaN
2000/04/20,0.125,0.142857,NaN,NaN,0.000,NaN,NaN,0.000000,0.0,0.111111,...,0.000000,0.0,0.090909,NaN,0.000000,0.000000,NaN,0.0,0.142857,NaN
2000/05/18,0.375,0.076923,NaN,NaN,0.000,NaN,NaN,0.333333,0.0,0.000000,...,0.000000,0.0,0.000000,NaN,0.000000,0.000000,NaN,0.0,0.000000,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021/08/19,0.000,0.090909,0.113636,0.0000,0.025,0.041667,0.000000,NaN,0.0,0.000000,...,0.416667,0.2,NaN,0.000000,NaN,0.000000,0.346154,0.0,0.428571,0.0
2021/09/16,0.000,0.000000,0.149733,0.0625,0.000,0.136364,0.111111,NaN,0.0,0.000000,...,0.000000,0.0,NaN,0.000000,NaN,0.000000,0.000000,0.0,0.000000,0.0
2021/10/14,0.000,0.000000,0.313492,0.0625,0.000,0.000000,0.000000,NaN,0.0,0.000000,...,0.181818,0.2,NaN,0.636364,NaN,0.000000,0.192308,0.0,0.000000,0.0


In [56]:
# num_estimates
df_temp = df_AnalystEstimates[["OFTIC", "STATPERS","NUMEST"]]
df_AnalystNum = pd.pivot_table(df_temp, values='NUMEST', index=["STATPERS"],
                    columns=['OFTIC'])
df_AnalystNum

OFTIC,A,AA,AAL,AAP,AAPL,ABBV,ABC,ABK,ABMD,ABS,...,XRAY,XRX,XTO,XYL,YHOO,YUM,ZBH,ZBRA,ZION,ZTS
STATPERS,,,,,,,,,,,,,,,,,,,,,
2000/01/20,7.0,12.0,NaN,NaN,13.0,NaN,NaN,5.0,NaN,8.0,...,2.0,9.0,10.0,NaN,11.0,6.0,NaN,3.0,7.0,NaN
2000/02/17,7.0,12.0,NaN,NaN,14.0,NaN,NaN,5.0,NaN,9.0,...,2.0,7.0,9.0,NaN,11.0,6.0,NaN,4.0,8.0,NaN
2000/03/16,8.0,14.0,NaN,NaN,15.0,NaN,NaN,4.0,1.0,9.0,...,2.0,7.0,8.0,NaN,11.0,6.0,NaN,4.0,7.0,NaN
2000/04/20,8.0,14.0,NaN,NaN,14.0,NaN,NaN,5.0,1.0,9.0,...,3.0,7.0,11.0,NaN,11.0,7.0,NaN,4.0,7.0,NaN
2000/05/18,8.0,13.0,NaN,NaN,14.0,NaN,NaN,3.0,1.0,9.0,...,2.0,6.0,10.0,NaN,13.0,5.0,NaN,4.0,5.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021/08/19,16.0,11.0,20.0,15.0,40.0,24.0,10.000000,NaN,6.0,1.5,...,12.0,5.0,NaN,12.0,NaN,23.0,26.0,10.0,21.0,13.0
2021/09/16,17.0,11.0,19.5,16.0,42.0,22.0,10.000000,NaN,6.0,1.5,...,12.0,5.0,NaN,12.0,NaN,22.0,26.0,10.0,21.0,13.0
2021/10/14,16.0,11.0,19.5,16.0,42.0,22.0,10.000000,NaN,6.0,1.5,...,11.0,5.0,NaN,11.0,NaN,22.0,26.0,10.0,22.0,12.0


In [57]:
#save to pickle
df_AnalystEstimates.to_pickle("FundamentalData/df_AnalystEstimates.pkl")
df_AnalystMedian.to_pickle("FundamentalData/df_AnalystMedian.pkl")
df_AnalystUp.to_pickle("FundamentalData/df_AnalystUp.pkl")
df_AnalystDown.to_pickle("FundamentalData/df_AnalystDown.pkl")
df_AnalystNum.to_pickle("FundamentalData/df_AnalystNum.pkl")
print("done")

done


# 4) Macro Data
- Downloaded from Capital IQ
- Except for PPI (credit @ Joseph Garcia Ben)

In [76]:
#convert xlsx to txt
#  open with encoding, sep & header
df_CPI = pd.read_csv("MacroData/US_CPI.txt", encoding = "utf-16", sep = "\t", header = None)
df_CPI

,0,1
0,Dec-31-2021,7.04
1,Nov-30-2021,6.81
2,Oct-31-2021,6.22
3,Sep-30-2021,5.39
4,Aug-31-2021,5.25
...,...,...
260,Apr-30-2000,3.07
261,Mar-31-2000,3.76
262,Feb-29-2000,3.22
263,Jan-31-2000,2.74


In [78]:
filenames = ["Dates_USAll",
            "Dates_USFed",
            "US_CPI",
            "US_FedFundsTargetRate",
            "US_GDP",
            "US_MedianHomeSalesPrice",
            "US_NonFarm",
            "US_NonFarmPayrolls",
            "US_PMI",
            "US_PPI",
            "US_PrivateHousingStarts",
            "US_Unemployment"]
for file in filenames:
    readname = "MacroData/" + file + ".txt"
    savename = "MacroData/df_" + file + ".pkl"
    df_temp = pd.read_csv(readname, encoding = "utf-16", sep = "\t", header = None)
    print(readname)
    print(df_temp[:3])
    df_temp.to_pickle(savename)
print("DONE")

MacroData/Dates_USAll.txt
                      0                              1
0  Nov-03-2004 10:00 AM      US: ISM Non-Manufacturing
1   Nov-10-2004 2:15 PM  US: FOMC - Fed Funds Rate (%)
2   Nov-16-2004 8:30 AM       US: PPI ex Food & Energy
MacroData/Dates_USFed.txt
                     0                              1
0  Nov-10-2004 2:15 PM  US: FOMC - Fed Funds Rate (%)
1  Dec-14-2004 2:15 PM  US: FOMC - Fed Funds Rate (%)
2  Feb-02-2005 2:15 PM  US: FOMC - Fed Funds Rate (%)
MacroData/US_CPI.txt
             0     1
0  Dec-31-2021  7.04
1  Nov-30-2021  6.81
2  Oct-31-2021  6.22
MacroData/US_FedFundsTargetRate.txt
             0     1
0  Feb-01-2022  0.25
1  Jan-01-2022  0.25
2  Dec-01-2021  0.25
MacroData/US_GDP.txt
             0         1
0  Dec-31-2021  19805962
1  Sep-30-2021  19478893
2  Jun-30-2021  19368310
MacroData/US_MedianHomeSalesPrice.txt
             0       1
0  Dec-31-2021  377700
1  Nov-30-2021  416100
2  Oct-31-2021  421500
MacroData/US_NonFarm.txt
           

# 5) Fundamental Ratios
- Data source: WRDS
https://wrds-www.wharton.upenn.edu/pages/get-data/financial-ratios-suite-wrds/financial-ratios-with-ibes-subscription/financial-ratios-firm-level-ibes/

## 5a) Initial ratios
- some were poorly filled, especially divyield, PEG_trailing, sale_nwc, quick_ratio
- 5b will replace these with alternative ratios

In [7]:
df_FundamentalRatios = pd.read_csv("FundamentalData/SPX_FundamentalRatios.csv")
df_FundamentalRatios

,permno,adate,qdate,public_date,evm,pe_exi,ps,npm,roe,debt_assets,de_ratio,quick_ratio,sale_nwc,ptb,PEG_trailing,divyield,TICKER,cusip
0,10104,19990531.0,19991130,20000131,23.669,25.486,15.247,0.155,0.413,0.470,0.888,1.834,4.325,40.279,0.158,NaN,ORCL,68389X10
1,10104,19990531.0,19991130,20000229,23.669,37.883,22.592,0.155,0.413,0.470,0.888,1.834,4.325,59.683,0.234,NaN,ORCL,68389X10
2,10104,19990531.0,19991130,20000331,23.669,39.828,23.752,0.155,0.413,0.470,0.888,1.834,4.325,62.748,0.246,NaN,ORCL,68389X10
3,10104,19990531.0,20000229,20000430,34.204,62.451,23.393,0.197,0.529,0.471,0.891,1.819,4.357,54.508,0.244,NaN,ORCL,68389X10
4,10104,19990531.0,20000229,20000531,34.204,56.152,20.805,0.197,0.529,0.471,0.891,1.819,4.357,30.870,0.219,NaN,ORCL,68389X10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
156627,93436,20191231.0,20200630,20200831,35.254,258.197,18.062,0.014,0.054,0.736,3.310,0.835,16.310,47.117,46.705,NaN,TSLA,88160R10
156628,93436,20191231.0,20200630,20200930,35.254,222.285,15.820,0.014,0.054,0.736,3.310,0.835,16.310,41.269,40.209,NaN,TSLA,88160R10
156629,93436,20191231.0,20200630,20201031,35.254,201.057,14.308,0.014,0.054,0.736,3.310,0.835,16.310,37.324,36.369,NaN,TSLA,88160R10
156630,93436,20191231.0,20200930,20201130,53.275,494.500,19.095,0.020,0.070,0.694,2.586,0.991,7.110,33.562,NaN,NaN,TSLA,88160R10


In [8]:
ratios = ["evm","pe_exi","ps", "npm", "roe", "debt_assets", "de_ratio","quick_ratio",  "ptb"]
for ratio in ratios:
    savename = "FundamentalData/df_" + ratio + ".pkl"
    df_temp = df_FundamentalRatios[["TICKER", "public_date",ratio]]
    df_save = pd.pivot_table(df_temp, values=ratio, index=["public_date"],
                    columns=['TICKER'])
    print(savename)
    print(df_save[:2])
    df_save.to_pickle(savename)
print("DONE")



FundamentalData/df_evm.pkl
TICKER        A      AA    AAG  AAL  AAP    AAPL    AAS  ABBV  ABC  ABI  ...  \
public_date                                                              ...   
20000131    NaN   8.936  9.348  NaN  NaN  22.570  9.263   NaN  NaN  NaN  ...   
20000229    NaN  12.501  8.473  NaN  NaN  19.266  9.256   NaN  NaN  NaN  ...   

TICKER         XTO  XYL     YHOO     YUM       Z  ZBH    ZBRA    ZION  ZMH  \
public_date                                                                  
20000131     9.082  NaN  302.645  10.211  12.439  NaN  10.355  14.710  NaN   
20000229     7.706  NaN  586.309   6.732  12.439  NaN  15.435  17.097  NaN   

TICKER       ZTS  
public_date       
20000131     NaN  
20000229     NaN  

[2 rows x 926 columns]
FundamentalData/df_pe_exi.pkl
TICKER        A      AA     AAG  AAL  AAP    AAPL     AAS  ABBV  ABC  ABI  \
public_date                                                                 
20000131    NaN  41.813  10.117  NaN  NaN  28.740  13.1

In [9]:
for ratio in ratios:
    df_temp = df_FundamentalRatios[["TICKER", "public_date",ratio]]
    filled = df_temp.count() / len(df_temp)
    print(ratio, filled[2])

evm 0.9947201082792788
pe_exi 0.981549108738955
ps 0.9981102201338169
npm 0.9981102201338169
roe 0.9655434393993565
debt_assets 0.9985315899688442
de_ratio 0.9985124367945247
quick_ratio 0.8704287757290975
ptb 0.9672416875223454


## 5b) Alternative ratios
- replace trailing PEGs with forward-looking
- quick ratio does not need replacing
- replace sale_nwc with cash_conversion

In [10]:
df_FundamentalRatios2 = pd.read_csv("FundamentalData/SPX_FundamentalRatios2.csv")
df_FundamentalRatios2

,permno,adate,qdate,public_date,curr_ratio,cash_conversion,PEG_1yrforward,PEG_ltgforward,TICKER,cusip
0,10104,19990531.0,19991130,20000131,1.834,58.838,0.801,1.045,ORCL,68389X10
1,10104,19990531.0,19991130,20000229,1.834,58.838,1.191,1.537,ORCL,68389X10
2,10104,19990531.0,19991130,20000331,1.834,58.838,0.876,1.584,ORCL,68389X10
3,10104,19990531.0,20000229,20000430,1.819,58.838,1.374,2.447,ORCL,68389X10
4,10104,19990531.0,20000229,20000531,1.819,58.838,1.235,2.242,ORCL,68389X10
...,...,...,...,...,...,...,...,...,...,...
156627,93436,20191231.0,20200630,20200831,1.182,22.445,NaN,NaN,TSLA,88160R10
156628,93436,20191231.0,20200630,20200930,1.182,22.445,NaN,NaN,TSLA,88160R10
156629,93436,20191231.0,20200630,20201031,1.182,22.445,NaN,NaN,TSLA,88160R10
156630,93436,20191231.0,20200930,20201130,1.329,18.662,NaN,NaN,TSLA,88160R10


In [11]:
ratios2 = ["curr_ratio","cash_conversion","PEG_1yrforward", "PEG_ltgforward"]
for ratio in ratios2:
    savename = "FundamentalData/df_" + ratio + ".pkl"
    df_temp = df_FundamentalRatios2[["TICKER", "public_date",ratio]]
    df_save = pd.pivot_table(df_temp, values=ratio, index=["public_date"],
                    columns=['TICKER'])
    print(savename)
    print(df_save[:2])
    df_save.to_pickle(savename)
print("DONE")



FundamentalData/df_curr_ratio.pkl
TICKER        A     AA  AAL  AAP   AAPL    AAS  ABBV  ABC  ABI   ABMD  ...  \
public_date                                                            ...   
20000131    NaN  1.539  NaN  NaN  2.766  1.447   NaN  NaN  NaN  4.976  ...   
20000229    NaN  1.598  NaN  NaN  2.665  1.498   NaN  NaN  NaN  4.226  ...   

TICKER         XRX    XTO  XYL   YHOO    YUM      Z  ZBH   ZBRA  ZMH  ZTS  
public_date                                                                
20000131     1.605  1.577  NaN  4.852  0.493  1.216  NaN  7.490  NaN  NaN  
20000229     1.508  1.529  NaN  4.919  0.374  1.216  NaN  8.027  NaN  NaN  

[2 rows x 823 columns]
FundamentalData/df_cash_conversion.pkl
TICKER        A      AA     AAG  AAL  AAP     AAS  ABBV  ABC  ABI  ABK  ...  \
public_date                                                             ...   
20000131    NaN  69.851  74.157  NaN  NaN  21.188   NaN  NaN  NaN  NaN  ...   
20000229    NaN  70.511  62.873  NaN  NaN  24.201

In [12]:
for ratio in ratios2:
    df_temp = df_FundamentalRatios2[["TICKER", "public_date",ratio]]
    filled = df_temp.count() / len(df_temp)
    print(ratio, filled[2])

curr_ratio 0.87034577864038
cash_conversion 0.7951376474794423
PEG_1yrforward 0.8867600490321262
PEG_ltgforward 0.8420948465192298


In [13]:
for ratio in ["quick_ratio","sale_nwc","PEG_trailing"]:
    df_temp = df_FundamentalRatios[["TICKER", "public_date",ratio]]
    filled = df_temp.count() / len(df_temp)
    print(ratio, filled[2])

quick_ratio 0.8704287757290975
sale_nwc 0.710404004290311
PEG_trailing 0.6075067674549262


## 5c) Cross-checking with Nakagawa 2020
- include any additional data from that study
- Use Div. payout ratio instead of div yield since the latter has 0% completion whereas the former has ~90%

In [14]:
df_FundamentalRatios3 = pd.read_csv("FundamentalData/SPX_FundamentalRatios3.csv")
df_FundamentalRatios3

,permno,adate,qdate,public_date,pcf,dpr,roa,aftret_invcapx,at_turn,accrual,TICKER
0,10104,1999/05/31,1999/11/30,2000/01/31,72.985,0.0,0.393,0.400,1.418,0.079,ORCL
1,10104,1999/05/31,1999/11/30,2000/02/29,108.144,0.0,0.393,0.400,1.418,0.079,ORCL
2,10104,1999/05/31,1999/11/30,2000/03/31,113.696,0.0,0.393,0.400,1.418,0.079,ORCL
3,10104,1999/05/31,2000/02/29,2000/04/30,91.705,0.0,0.439,0.400,1.407,0.084,ORCL
4,10104,1999/05/31,2000/02/29,2000/05/31,81.560,0.0,0.439,0.400,1.407,0.084,ORCL
...,...,...,...,...,...,...,...,...,...,...,...
156627,93436,2019/12/31,2020/06/30,2020/08/31,171.660,0.0,0.100,0.061,0.722,0.067,TSLA
156628,93436,2019/12/31,2020/06/30,2020/09/30,150.352,0.0,0.100,0.061,0.722,0.067,TSLA
156629,93436,2019/12/31,2020/06/30,2020/10/31,135.979,0.0,0.100,0.061,0.722,0.067,TSLA
156630,93436,2019/12/31,2020/09/30,2020/11/30,123.713,0.0,0.111,0.068,0.725,0.101,TSLA


In [15]:
ratios3 = ["pcf","dpr","roa", "aftret_invcapx", "at_turn", "accrual"]
for ratio in ratios3:
    savename = "FundamentalData/df_" + ratio + ".pkl"
    df_temp = df_FundamentalRatios3[["TICKER", "public_date",ratio]]
    df_save = pd.pivot_table(df_temp, values=ratio, index=["public_date"],
                    columns=['TICKER'])
    print(savename)
    print(df_save[:2])
    df_save.to_pickle(savename)
print("DONE")



FundamentalData/df_pcf.pkl
TICKER        A      AA    AAG  AAL  AAP    AAPL      AAS  ABBV  ABC  ABI  \
public_date                                                                 
2000/01/31  NaN  11.594  2.182  NaN  NaN  20.953  309.261   NaN  NaN  NaN   
2000/02/29  NaN  11.319  2.104  NaN  NaN  19.486   25.017   NaN  NaN  NaN   

TICKER       ...    XTO  XYL     YHOO    YUM      Z  ZBH    ZBRA   ZION  ZMH  \
public_date  ...                                                               
2000/01/31   ...  2.638  NaN  525.144  7.061  1.382  NaN  45.736 -2.882  NaN   
2000/02/29   ...  3.156  NaN  396.082  7.265  1.310  NaN  70.922 -2.422  NaN   

TICKER       ZTS  
public_date       
2000/01/31   NaN  
2000/02/29   NaN  

[2 rows x 926 columns]
FundamentalData/df_dpr.pkl
TICKER        A     AA    AAG  AAL  AAP  AAPL  AAS  ABBV  ABC    ABK  ...  \
public_date                                                           ...   
2000/01/31  NaN  0.317  0.062  NaN  NaN   0.0  0.0   NaN  NaN 

In [16]:
for ratio in ratios3:
    df_temp = df_FundamentalRatios3[["TICKER", "public_date",ratio]]
    filled = df_temp.count() / len(df_temp)
    print(ratio, filled[2])

pcf 0.9978356913019051
dpr 0.8856300117472803
roa 0.9978803820419837
aftret_invcapx 0.9843773941467899
at_turn 0.9969865672404107
accrual 0.9980463762194187


# 6) Market Data
- From OptionMetrics. Updated annually, so we'll get 2021 data after 17 Feb. Link: https://wrds-www.wharton.upenn.edu/pages/get-data/optionmetrics/ivy-db-us/securities/security-prices/
- More comprehensive is compustat: https://wrds-www.wharton.upenn.edu/pages/get-data/compustat-capital-iq-standard-poors/compustat/north-america-daily/security-daily/
- daily granularity for OptionMetrics & Compustat. Monthly granularity is also available for Compustat
- Use the data in 6c!!

# 6a) OptionMetrics - daily
- shrout = shares outstanding (not sure about the units)
- In general, Compustat is better!! Use 6c for monthly data (we can switch to daily later if needed)

In [49]:
df_PriceData = pd.read_csv("MarketData/SPX_PriceData.csv")
df_PriceData

,secid,date,cusip,ticker,index_flag,industry_group,low,high,open,close,volume,return,shrout
0,5859,2000/03/31,82691210,SIG,0,NaN,25.0000,26.0000,25.0000,25.5000,252100,0.030303,23631
1,5859,2000/03/27,82691210,SIG,0,NaN,25.5625,25.9375,25.7500,25.5625,46500,-0.012077,23631
2,5859,2000/03/21,82691210,SIG,0,NaN,24.4375,24.7500,24.6250,24.7500,89800,0.007634,23631
3,5859,2000/03/15,82691210,SIG,0,NaN,24.0000,24.7500,24.6875,24.0625,17500,-0.025316,23631
4,5859,2000/03/10,82691210,SIG,0,NaN,23.1250,24.6875,23.1250,24.2500,44100,0.057221,23631
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3027745,214905,2020/11/23,92556V10,VTRS,0,NaN,16.5000,17.2500,17.2355,16.7700,20717971,-0.021016,516952
3027746,214905,2020/11/20,92556V10,VTRS,0,NaN,17.0000,18.4200,18.4000,17.1300,38015407,-0.054114,516952
3027747,214905,2020/11/19,92556V10,VTRS,0,NaN,15.9900,18.2000,16.0000,18.1100,35794059,0.118592,516952
3027748,214905,2020/11/18,92556V10,VTRS,0,NaN,15.9500,16.9900,16.4600,16.1900,31517441,-0.009180,0


In [48]:
df_PriceData["industry_group"].unique()

array([ nan, 130., 843., 725., 821., 734., 911., 325., 722., 634., 121.,
       842., 413., 124., 410., 411., 333., 522., 826., 760., 346., 732.,
       727., 125., 913., 419., 414., 133., 726., 769., 446., 111., 524.,
       841., 442., 710., 768., 622., 767., 834., 912., 511., 422., 737.,
       520., 112., 123., 515., 830., 824., 823., 312., 745., 711., 110.,
       844., 733., 851., 510., 731., 835., 521., 815., 122., 430., 424.,
       611., 432., 528., 756., 770., 423., 846., 320., 447., 840., 820.])

In [50]:
np.sort(df_PriceData["date"].unique())

array(['2000/01/03', '2000/01/04', '2000/01/05', ..., '2020/12/29',
       '2020/12/30', '2020/12/31'], dtype=object)

## 6b) Compustat - Daily
- Use monthly first for a speedy prototype. We also cannot rebalance daily without incurring huge transaction costs

In [59]:
df_PriceData = pd.read_csv("MarketData/SPX_PriceData_CompustatDaily.csv")
df_PriceData

C:\Users\pyliu\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (1,4) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,gvkey,iid,datadate,tic,cusip,cshoc,cshtrd,prccd,prcod,ggroup,gind,gsector,gsubind
0,1045,01,2000/01/03,AAMRQ,001765106,148083000.0,566200.0,67.3750,NaN,2030.0,203020.0,20.0,20302010.0
1,1045,01,2000/01/04,AAMRQ,001765106,148083000.0,811800.0,65.8750,NaN,2030.0,203020.0,20.0,20302010.0
2,1045,01,2000/01/05,AAMRQ,001765106,148083000.0,665500.0,63.8125,NaN,2030.0,203020.0,20.0,20302010.0
3,1045,01,2000/01/06,AAMRQ,001765106,148083000.0,1034300.0,62.1250,NaN,2030.0,203020.0,20.0,20302010.0
4,1045,01,2000/01/07,AAMRQ,001765106,148083000.0,662100.0,63.6875,NaN,2030.0,203020.0,20.0,20302010.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4243078,316056,01,2021/12/27,ALLE,G0176J109,89696000.0,532553.0,129.5800,127.98,2010.0,201020.0,20.0,20102010.0
4243079,316056,01,2021/12/28,ALLE,G0176J109,89696000.0,284253.0,131.0600,129.45,2010.0,201020.0,20.0,20102010.0
4243080,316056,01,2021/12/29,ALLE,G0176J109,89696000.0,319740.0,132.3700,131.04,2010.0,201020.0,20.0,20102010.0
4243081,316056,01,2021/12/30,ALLE,G0176J109,89696000.0,360149.0,131.5600,132.90,2010.0,201020.0,20.0,20102010.0


In [65]:
df_sorted = df_PriceData.sort_values("datadate")

In [71]:
df_PriceData.to_pickle("MarketData/SPX_PriceData_CompustatDaily.pkl")

## 6c) Compustat Monthly
- Use this!

In [58]:
df_PriceData = pd.read_csv("MarketData/SPX_PriceData_CompustatMonthly.csv")
df_PriceData

,gvkey,iid,datadate,tic,cusip,cshtrm,prccm,trfm,trt1m,cshom,ggroup,gind,gsector,gsubind
0,1045,01,2000/01/31,AAMRQ,001765106,22341200.0,53.8125,1.2935,-19.6828,148083000.0,2030.0,203020.0,20.0,20302010.0
1,1045,01,2000/02/29,AAMRQ,001765106,30739100.0,52.8750,1.2935,-1.7422,148083000.0,2030.0,203020.0,20.0,20302010.0
2,1045,01,2000/03/31,AAMRQ,001765106,38855400.0,31.8750,2.7140,26.4841,148583000.0,2030.0,203020.0,20.0,20302010.0
3,1045,01,2000/04/30,AAMRQ,001765106,26060700.0,34.0625,2.7140,6.8627,148583000.0,2030.0,203020.0,20.0,20302010.0
4,1045,01,2000/05/31,AAMRQ,001765106,28663100.0,28.5000,2.7140,-16.3303,149685000.0,2030.0,203020.0,20.0,20302010.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
174861,316056,01,2021/08/31,ALLE,G0176J109,12720673.0,143.9900,1.0683,5.4100,89696000.0,2010.0,201020.0,20.0,20102010.0
174862,316056,01,2021/09/30,ALLE,G0176J109,16997572.0,132.1800,1.0712,-7.9519,89696000.0,2010.0,201020.0,20.0,20102010.0
174863,316056,01,2021/10/31,ALLE,G0176J109,16003363.0,128.3000,1.0712,-2.9354,89696000.0,2010.0,201020.0,20.0,20102010.0
174864,316056,01,2021/11/30,ALLE,G0176J109,11978831.0,123.6400,1.0712,-3.6321,89696000.0,2010.0,201020.0,20.0,20102010.0


# 7) Create Machine Learning data
- Fundamental: load individual dataframes, melt then merge
- Macro: load individual dataframes, map onto correct
- Market:

## 7a) Fundamental ratios

In [22]:
fundamental_ratios = ListUnion(ratios, ratios2)
fundamental_ratios = ListUnion(fundamental_ratios, ratios3)
fundamental_ratios

['debt_assets',
 'de_ratio',
 'evm',
 'pe_exi',
 'accrual',
 'quick_ratio',
 'at_turn',
 'curr_ratio',
 'PEG_ltgforward',
 'dpr',
 'roa',
 'roe',
 'npm',
 'cash_conversion',
 'PEG_1yrforward',
 'ps',
 'ptb',
 'pcf',
 'aftret_invcapx']

In [183]:
i=0
readname = "FundamentalData/df_" + fundamental_ratios[i] + ".pkl"
df_temp = pd.read_pickle(readname)
if df_temp.index.dtype == "int64":
    df_temp.index = pd.to_datetime(df_temp.index.astype(str), format='%Y%m%d') #date format
else:
    df_temp.index = pd.to_datetime(list(df_temp.index))
df_temp.index.rename("date", inplace = True)
df_fundamental = pd.melt(df_temp, var_name = "ticker", value_name = fundamental_ratios[i], ignore_index = False)

for ratio in fundamental_ratios[1:]:
    readname = "FundamentalData/df_" + ratio + ".pkl"
    df_temp = pd.read_pickle(readname)
    if df_temp.index.dtype == "int64":
        df_temp.index = pd.to_datetime(df_temp.index.astype(str), format='%Y%m%d') #date format
    else:
        df_temp.index = pd.to_datetime(list(df_temp.index))
    df_temp.index.rename("date", inplace = True)
    df_melt = pd.melt(df_temp, var_name = "ticker", value_name = ratio, ignore_index = False)
    df_fundamental = df_fundamental.merge(df_melt, on = ["date", "ticker"], how = "outer")
df_fundamental.to_pickle("df_fundamental_all.pkl")


## 7b) Macro

In [184]:
df_fundamental["month"] = df_fundamental.index
df_fundamental["month"] = df_fundamental["month"].dt.to_period('M')
df_macro = df_fundamental
df_macro

,ticker,debt_assets,de_ratio,evm,pe_exi,accrual,quick_ratio,at_turn,curr_ratio,PEG_ltgforward,...,roa,roe,npm,cash_conversion,PEG_1yrforward,ps,ptb,pcf,aftret_invcapx,month
date,,,,,,,,,,,,,,,,,,,,,
2000-01-31,A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2000-01
2000-02-29,A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2000-02
2000-03-31,A,0.370,0.587,0.000,76.471,-0.010,1.364,1.431,2.258,3.466,...,0.243,0.142,0.063,158.786,3.858,5.599,10.479,101.970,0.142,2000-03
2000-04-30,A,0.370,0.587,0.000,65.165,-0.010,1.364,1.431,2.258,2.954,...,0.243,0.142,0.063,158.786,3.288,4.771,8.930,86.895,0.142,2000-04
2000-05-31,A,0.370,0.587,0.000,54.136,-0.010,1.364,1.431,2.258,2.480,...,0.243,0.142,0.063,158.786,2.613,3.964,7.418,72.188,0.142,2000-05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-08-31,ZTS,0.765,3.263,25.850,47.507,0.027,2.271,0.536,3.196,NaN,...,0.231,0.525,0.254,322.952,NaN,11.972,22.526,39.333,0.196,2020-08
2020-09-30,ZTS,0.765,3.263,25.850,49.071,0.027,2.271,0.536,3.196,NaN,...,0.231,0.525,0.254,322.952,NaN,12.366,23.268,40.628,0.196,2020-09
2020-10-31,ZTS,0.765,3.263,25.850,47.047,0.027,2.271,0.536,3.196,NaN,...,0.231,0.525,0.254,322.952,NaN,11.856,22.308,38.953,0.196,2020-10


In [23]:
macro = ["CPI", "FedFundsTargetRate", "GDP", "MedianHomeSalesPrice", 
         "NonFarmPayrolls", "PMI", "PPI", "PrivateHousingStarts", "Unemployment"]

In [185]:
for ratio in macro:
    readname = "MacroData/df_US_" + ratio + ".pkl"
    df_temp = pd.read_pickle(readname)
    if ratio == "PPI":   #PPI had different format
        df_temp = df_temp[1:]
        df_temp.columns = ["year", "m", "month", "PPI"]
        df_temp = df_temp[["month", "PPI"]]
        df_temp["month"] = pd.to_datetime(df_temp["month"]).dt.to_period('M')
    else:
        df_temp.columns = ["date", ratio]
        df_temp["date"] = pd.to_datetime(df_temp["date"])   #format date
        df_temp["month"] = df_temp['date'].dt.to_period('M')   #nearest month
        df_temp = df_temp[["month", ratio]]
    df_macro = df_macro.merge(df_temp, on = ["month"], how = "inner")

df_macro.to_pickle("df_fundamental_macro.pkl")

,ticker,debt_assets,de_ratio,evm,pe_exi,accrual,quick_ratio,at_turn,curr_ratio,PEG_ltgforward,...,month,CPI,FedFundsTargetRate,GDP,MedianHomeSalesPrice,NonFarmPayrolls,PMI,PPI,PrivateHousingStarts,Unemployment
0,A,0.370,0.587,0.000,76.471,-0.010,1.364,1.431,2.258,3.466,...,2000-03,3.76,5.83871,12935252,165100,131596000,54.9,130.8,1.89,4.0
1,AA,0.544,1.470,12.501,24.911,0.068,1.060,0.945,1.598,NaN,...,2000-03,3.76,5.83871,12935252,165100,131596000,54.9,130.8,1.89,4.0
2,AAG,0.930,13.325,8.473,10.304,0.034,NaN,0.086,NaN,NaN,...,2000-03,3.76,5.83871,12935252,165100,131596000,54.9,130.8,1.89,4.0
3,AAL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2000-03,3.76,5.83871,12935252,165100,131596000,54.9,130.8,1.89,4.0
4,AAP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2000-03,3.76,5.83871,12935252,165100,131596000,54.9,130.8,1.89,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77779,ZBH,0.511,1.047,17.772,-205.450,0.055,0.954,0.289,1.799,NaN,...,2020-12,1.36,0.25000,18767778,365300,142503000,60.5,200.5,1.76,6.7
77780,ZBRA,0.618,1.617,17.961,NaN,0.068,0.452,0.908,0.782,NaN,...,2020-12,1.36,0.25000,18767778,365300,142503000,60.5,200.5,1.76,6.7
77781,ZION,0.898,8.826,11.025,18.485,0.006,NaN,0.041,NaN,NaN,...,2020-12,1.36,0.25000,18767778,365300,142503000,60.5,200.5,1.76,6.7
77782,ZMH,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2020-12,1.36,0.25000,18767778,365300,142503000,60.5,200.5,1.76,6.7


## 7c) Analyst estimates

In [197]:
analyst = ["Down", "Up", "Estimates", "Median", "Num"]

In [221]:
ratio = "Estimates"
readname = "FundamentalData/df_Analyst" + ratio + ".pkl"
df_temp = pd.read_pickle(readname)
df_temp

,OFTIC,TICKER,CUSIP,CNAME,STATPERS,NUMEST,NUMUP4W,NUMDOWN4W,NUMUP1M,NUMDOWN1M,MEANPTG,MEDPTG,STDEV,PTGHIGH,PTGLOW,CURR
0,A,@F9I,SI496526,ALFESCA,2006/03/16,1,0,0,0,0,5.100,5.1,NaN,5.1,5.1,ISK
1,A,@F9I,SI496526,ALFESCA,2006/04/20,1,1,0,1,0,5.400,5.4,NaN,5.4,5.4,ISK
2,A,@F9I,SI496526,ALFESCA,2006/05/18,1,0,0,0,0,5.400,5.4,NaN,5.4,5.4,ISK
3,A,@F9I,SI496526,ALFESCA,2006/06/15,1,0,0,0,0,5.400,5.4,NaN,5.4,5.4,ISK
4,A,@F9I,SI496526,ALFESCA,2006/07/20,1,0,0,0,0,5.400,5.4,NaN,5.4,5.4,ISK
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
228342,ZTS,ZOTS,98978V10,ZOETIS INC,2021/08/19,13,9,0,9,0,225.000,227.0,14.259,240.0,184.0,USD
228343,ZTS,ZOTS,98978V10,ZOETIS INC,2021/09/16,13,0,0,0,0,225.000,227.0,14.259,240.0,184.0,USD
228344,ZTS,ZOTS,98978V10,ZOETIS INC,2021/10/14,12,0,0,0,0,224.583,226.0,14.811,240.0,184.0,USD
228345,ZTS,ZOTS,98978V10,ZOETIS INC,2021/11/18,11,7,0,7,0,236.545,245.0,21.398,258.0,184.0,USD


In [222]:
df_temp["NUMUP4W"] = df_temp["NUMUP4W"]/df_temp["NUMEST"]   #use 4 weeks since there are granularity issues with 1M (e.g. NUMDOWN1M > NUMEST for index 180834)
df_temp["NUMDOWN4W"] = df_temp["NUMDOWN4W"]/df_temp["NUMEST"]
df_temp["STATPERS"] = pd.to_datetime(df_temp["STATPERS"])
df_temp["month"] = df_temp["STATPERS"].dt.to_period("M")
df_narrow = df_temp[["OFTIC", "month", "MEDPTG", "NUMUP4W", "NUMDOWN4W","NUMEST"]]
df_narrow.columns = ["ticker", "month", "MEDPTG", "PCTUP4W", "PCTDOWN4W", "NUMEST"]
df_narrow

,ticker,month,MEDPTG,PCTUP4W,PCTDOWN4W,NUMEST
0,A,2006-03,5.1,0.000000,0.0,1
1,A,2006-04,5.4,1.000000,0.0,1
2,A,2006-05,5.4,0.000000,0.0,1
3,A,2006-06,5.4,0.000000,0.0,1
4,A,2006-07,5.4,0.000000,0.0,1
...,...,...,...,...,...,...
228342,ZTS,2021-08,227.0,0.692308,0.0,13
228343,ZTS,2021-09,227.0,0.000000,0.0,13
228344,ZTS,2021-10,226.0,0.000000,0.0,12
228345,ZTS,2021-11,245.0,0.636364,0.0,11


In [223]:
df_FundamentalMacroAnalyst = df_macro.merge(df_narrow, on = ["ticker", "month"], how = "inner")
df_FundamentalMacroAnalyst

,ticker,debt_assets,de_ratio,evm,pe_exi,accrual,quick_ratio,at_turn,curr_ratio,PEG_ltgforward,...,MedianHomeSalesPrice,NonFarmPayrolls,PMI,PPI,PrivateHousingStarts,Unemployment,MEDPTG,PCTUP4W,PCTDOWN4W,NUMEST
0,A,0.370,0.587,0.000,76.471,-0.010,1.364,1.431,2.258,3.466,...,165100,131596000,54.9,130.8,1.89,4.0,172.5,0.750000,0.000000,8
1,AA,0.544,1.470,12.501,24.911,0.068,1.060,0.945,1.598,NaN,...,165100,131596000,54.9,130.8,1.89,4.0,97.5,0.000000,0.142857,14
2,AAPL,0.442,0.793,19.266,0.328,0.058,2.656,1.192,2.665,0.020,...,165100,131596000,54.9,130.8,1.89,4.0,139.0,0.000000,0.000000,15
3,ABK,0.822,4.621,23.567,11.688,0.013,NaN,0.046,NaN,0.779,...,165100,131596000,54.9,130.8,1.89,4.0,74.0,0.000000,0.000000,4
4,ABMD,0.208,0.263,-16.591,-20.050,0.045,3.688,0.740,4.226,NaN,...,165100,131596000,54.9,130.8,1.89,4.0,100.0,0.000000,0.000000,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67918,YUM,2.356,-1.737,19.277,31.558,0.039,1.148,0.942,1.309,NaN,...,365300,142503000,60.5,200.5,1.76,6.7,106.5,0.227273,0.000000,22
67919,ZBH,0.511,1.047,17.772,-205.450,0.055,0.954,0.289,1.799,NaN,...,365300,142503000,60.5,200.5,1.76,6.7,165.0,0.230769,0.000000,26
67920,ZBRA,0.618,1.617,17.961,NaN,0.068,0.452,0.908,0.782,NaN,...,365300,142503000,60.5,200.5,1.76,6.7,361.5,0.300000,0.000000,10
67921,ZION,0.898,8.826,11.025,18.485,0.006,NaN,0.041,NaN,NaN,...,365300,142503000,60.5,200.5,1.76,6.7,41.0,0.409091,0.000000,22


In [224]:
df_FundamentalMacroAnalyst.to_pickle("df_FundamentalMacroAnalyst.pkl")
print("done")

done


## 7d) Market data

In [273]:
readname = "MarketData/SPX_PriceData_CompustatMonthly.csv"
df_temp = pd.read_csv(readname)
df_temp

,gvkey,iid,datadate,tic,cusip,ajexm,cshtrm,curcdm,prccm,trfm,trt1m,cshom,ggroup,gind,gsector,gsubind
0,1045,01,20000131,AAMRQ,001765106,1.0,22341200.0,USD,53.8125,1.2935,-19.6828,148083000.0,2030.0,203020.0,20.0,20302010.0
1,1045,01,20000229,AAMRQ,001765106,1.0,30739100.0,USD,52.8750,1.2935,-1.7422,148083000.0,2030.0,203020.0,20.0,20302010.0
2,1045,01,20000331,AAMRQ,001765106,1.0,38855400.0,USD,31.8750,2.7140,26.4841,148583000.0,2030.0,203020.0,20.0,20302010.0
3,1045,01,20000430,AAMRQ,001765106,1.0,26060700.0,USD,34.0625,2.7140,6.8627,148583000.0,2030.0,203020.0,20.0,20302010.0
4,1045,01,20000531,AAMRQ,001765106,1.0,28663100.0,USD,28.5000,2.7140,-16.3303,149685000.0,2030.0,203020.0,20.0,20302010.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
174861,316056,01,20210831,ALLE,G0176J109,1.0,12720673.0,USD,143.9900,1.0683,5.4100,89696000.0,2010.0,201020.0,20.0,20102010.0
174862,316056,01,20210930,ALLE,G0176J109,1.0,16997572.0,USD,132.1800,1.0712,-7.9519,89696000.0,2010.0,201020.0,20.0,20102010.0
174863,316056,01,20211031,ALLE,G0176J109,1.0,16003363.0,USD,128.3000,1.0712,-2.9354,89696000.0,2010.0,201020.0,20.0,20102010.0
174864,316056,01,20211130,ALLE,G0176J109,1.0,11978831.0,USD,123.6400,1.0712,-3.6321,89696000.0,2010.0,201020.0,20.0,20102010.0


In [274]:
df_temp["month"] = pd.to_datetime(df_temp["datadate"].astype(str), format='%Y%m%d').dt.to_period("M")
df_temp.columns = ["gvkey", "iid", "date", "ticker", "cusip", "adjustment_factor","volume", "curr","price_close", "total_return_factor", "total_return","shares_outstanding", "ggroup","gind", "gsector", "gsubind","month"]
df_temp["mcap"] = df_temp["price_close"] * df_temp["shares_outstanding"]
df_temp["price_adjusted"] = df_temp["price_close"] * df_temp["total_return_factor"]/df_temp["adjustment_factor"]
df_temp

,gvkey,iid,date,ticker,cusip,adjustment_factor,volume,curr,price_close,total_return_factor,total_return,shares_outstanding,ggroup,gind,gsector,gsubind,month,mcap,price_adjusted
0,1045,01,20000131,AAMRQ,001765106,1.0,22341200.0,USD,53.8125,1.2935,-19.6828,148083000.0,2030.0,203020.0,20.0,20302010.0,2000-01,7.968716e+09,69.606469
1,1045,01,20000229,AAMRQ,001765106,1.0,30739100.0,USD,52.8750,1.2935,-1.7422,148083000.0,2030.0,203020.0,20.0,20302010.0,2000-02,7.829889e+09,68.393813
2,1045,01,20000331,AAMRQ,001765106,1.0,38855400.0,USD,31.8750,2.7140,26.4841,148583000.0,2030.0,203020.0,20.0,20302010.0,2000-03,4.736083e+09,86.508750
3,1045,01,20000430,AAMRQ,001765106,1.0,26060700.0,USD,34.0625,2.7140,6.8627,148583000.0,2030.0,203020.0,20.0,20302010.0,2000-04,5.061108e+09,92.445625
4,1045,01,20000531,AAMRQ,001765106,1.0,28663100.0,USD,28.5000,2.7140,-16.3303,149685000.0,2030.0,203020.0,20.0,20302010.0,2000-05,4.266022e+09,77.349000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
174861,316056,01,20210831,ALLE,G0176J109,1.0,12720673.0,USD,143.9900,1.0683,5.4100,89696000.0,2010.0,201020.0,20.0,20102010.0,2021-08,1.291533e+10,153.824517
174862,316056,01,20210930,ALLE,G0176J109,1.0,16997572.0,USD,132.1800,1.0712,-7.9519,89696000.0,2010.0,201020.0,20.0,20102010.0,2021-09,1.185602e+10,141.591216
174863,316056,01,20211031,ALLE,G0176J109,1.0,16003363.0,USD,128.3000,1.0712,-2.9354,89696000.0,2010.0,201020.0,20.0,20102010.0,2021-10,1.150800e+10,137.434960
174864,316056,01,20211130,ALLE,G0176J109,1.0,11978831.0,USD,123.6400,1.0712,-3.6321,89696000.0,2010.0,201020.0,20.0,20102010.0,2021-11,1.109001e+10,132.443168


In [275]:
df_narrow = df_temp[["ticker","month", "volume", "price_adjusted", "shares_outstanding", "ggroup", "gind", "gsector", "mcap"]]
df_narrow

,ticker,month,volume,price_adjusted,shares_outstanding,ggroup,gind,gsector,mcap
0,AAMRQ,2000-01,22341200.0,69.606469,148083000.0,2030.0,203020.0,20.0,7.968716e+09
1,AAMRQ,2000-02,30739100.0,68.393813,148083000.0,2030.0,203020.0,20.0,7.829889e+09
2,AAMRQ,2000-03,38855400.0,86.508750,148583000.0,2030.0,203020.0,20.0,4.736083e+09
3,AAMRQ,2000-04,26060700.0,92.445625,148583000.0,2030.0,203020.0,20.0,5.061108e+09
4,AAMRQ,2000-05,28663100.0,77.349000,149685000.0,2030.0,203020.0,20.0,4.266022e+09
...,...,...,...,...,...,...,...,...,...
174861,ALLE,2021-08,12720673.0,153.824517,89696000.0,2010.0,201020.0,20.0,1.291533e+10
174862,ALLE,2021-09,16997572.0,141.591216,89696000.0,2010.0,201020.0,20.0,1.185602e+10
174863,ALLE,2021-10,16003363.0,137.434960,89696000.0,2010.0,201020.0,20.0,1.150800e+10
174864,ALLE,2021-11,11978831.0,132.443168,89696000.0,2010.0,201020.0,20.0,1.109001e+10


In [276]:
df_all = df_FundamentalMacroAnalyst.merge(df_narrow, on = ["ticker", "month"], how = "inner")
df_all

,ticker,debt_assets,de_ratio,evm,pe_exi,accrual,quick_ratio,at_turn,curr_ratio,PEG_ltgforward,...,PCTUP4W,PCTDOWN4W,NUMEST,volume,price_adjusted,shares_outstanding,ggroup,gind,gsector,mcap
0,A,0.370,0.587,0.000,76.471,-0.010,1.364,1.431,2.258,3.466,...,0.750000,0.000000,8,41201000.0,104.000000,4.520000e+08,3520.0,352030.0,35.0,4.700800e+10
1,AAPL,0.442,0.793,19.266,0.328,0.058,2.656,1.192,2.665,0.020,...,0.000000,0.000000,15,77654500.0,1.329265,1.622960e+08,4520.0,452020.0,45.0,2.204183e+10
2,ABMD,0.208,0.263,-16.591,-20.050,0.045,3.688,0.740,4.226,NaN,...,0.000000,0.000000,1,4729700.0,20.250000,1.020400e+07,3510.0,351010.0,35.0,4.132620e+08
3,ABT,0.487,0.948,14.035,22.412,0.036,1.090,0.952,1.421,1.836,...,0.000000,0.076923,13,96184900.0,78.506831,1.547694e+09,3510.0,351010.0,35.0,5.445948e+10
4,ACAS,0.212,0.268,15.995,3.732,-0.229,NaN,0.100,NaN,0.251,...,0.000000,0.000000,4,1570600.0,30.741813,1.822400e+07,4020.0,402030.0,40.0,4.624340e+08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58933,YUM,2.356,-1.737,19.277,31.558,0.039,1.148,0.942,1.309,NaN,...,0.227273,0.000000,22,36685555.0,202.866072,3.016680e+08,2530.0,253010.0,25.0,3.274908e+10
58934,ZBH,0.511,1.047,17.772,-205.450,0.055,0.954,0.289,1.799,NaN,...,0.230769,0.000000,26,19011810.0,166.494245,2.072780e+08,3510.0,351010.0,35.0,3.193947e+10
58935,ZBRA,0.618,1.617,17.961,NaN,0.068,0.452,0.908,0.782,NaN,...,0.300000,0.000000,10,7394890.0,384.330000,5.331600e+07,4520.0,452030.0,45.0,2.049094e+10
58936,ZION,0.898,8.826,11.025,18.485,0.006,NaN,0.041,NaN,NaN,...,0.409091,0.000000,22,35218731.0,151.514376,1.640120e+08,4010.0,401010.0,40.0,7.124681e+09


In [278]:
df_all.columns

Index(['ticker', 'debt_assets', 'de_ratio', 'evm', 'pe_exi', 'accrual',
       'quick_ratio', 'at_turn', 'curr_ratio', 'PEG_ltgforward', 'dpr', 'roa',
       'roe', 'npm', 'cash_conversion', 'PEG_1yrforward', 'ps', 'ptb', 'pcf',
       'aftret_invcapx', 'month', 'CPI', 'FedFundsTargetRate', 'GDP',
       'MedianHomeSalesPrice', 'NonFarmPayrolls', 'PMI', 'PPI',
       'PrivateHousingStarts', 'Unemployment', 'MEDPTG', 'PCTUP4W',
       'PCTDOWN4W', 'NUMEST', 'volume', 'price_adjusted', 'shares_outstanding',
       'ggroup', 'gind', 'gsector', 'mcap'],
      dtype='object')

In [279]:
df_all.drop("month", axis = 1)
df_all.to_pickle("df_all_unnormalised.pkl")
print("done")

done


In [280]:
df_all.count()/len(df_all)

ticker                  1.000000
debt_assets             0.951661
de_ratio                0.951661
evm                     0.947962
pe_exi                  0.935033
accrual                 0.951610
quick_ratio             0.819743
at_turn                 0.950881
curr_ratio              0.819675
PEG_ltgforward          0.839696
dpr                     0.870186
roa                     0.951576
roe                     0.925583
npm                     0.951763
cash_conversion         0.756863
PEG_1yrforward          0.875598
ps                      0.951763
ptb                     0.927025
pcf                     0.951610
aftret_invcapx          0.941888
month                   1.000000
CPI                     1.000000
FedFundsTargetRate      1.000000
GDP                     1.000000
MedianHomeSalesPrice    1.000000
NonFarmPayrolls         1.000000
PMI                     1.000000
PPI                     1.000000
PrivateHousingStarts    1.000000
Unemployment            1.000000
MEDPTG    